# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

2024-01-03 17:44:41.795926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works, mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [2]:
path_to_file = 'DATA/shakespeare.txt'

### Understanding unique characters

In [3]:
def get_vocab(file, lower=False):
    with open(file, 'r') as fopen:
        data = fopen.read()
    if lower:
        data = data.lower()
    vocab = sorted(set(data))
    return data, vocab

In [4]:
text, vocab = get_vocab(path_to_file)

In [5]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [6]:
vocab

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}']

In [7]:
print(len(text), len(vocab))

5445609 84


Our datafile consists of 5445609 characters in total as combinations of 84 unique characters.

## Text Processing

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [8]:
def embed_to_onehot(data, vocab):
    onehot = np.zeros((len(data), len(vocab)), dtype = np.float32)
    for i in range(len(data)):
        onehot[i, vocab.index(data[i])] = 1.0
    return onehot

onehot = embed_to_onehot(text, vocab)
print(onehot.shape) # too many unique chars for one hot encoding them all

(5445609, 84)


There are too many unique characters and things will get out of hand really fast if we try to one-hot-encode them all. We can use a differend kind of mapping to go back and forth from characters to numerics.

In [9]:
# Use dictionary comprehension to construct a character to numeric-index kind of map
char_to_idx = {char:idx for idx, char in enumerate(vocab)}

# The opposite is as simple as an array of characters, each one with its own numeric-index location
idx_to_char = np.array(vocab)

# Numeric index of the letter 'u'
print(char_to_idx.get('u'))

# Letter that corresponds to index 59
print(idx_to_char[59])

76
d


In [10]:
encoded_text = np.array([char_to_idx[c] for c in text])

In [11]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

Here's what this char-to-numeric map looks like:

In [12]:
sample = text[:20]

In [13]:
sample

'\n                   '

In [14]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence should be. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, we should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [15]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [16]:
line = "From fairest creatures we desire increase"

In [17]:
len(line)

41

In [18]:
lines = """
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
"""

In [19]:
len(lines)

133

We can try and use a sequence length of 120 characters to capture the overall structure of the text.

In [20]:
seq_len = 120

In [21]:
total_num_seq = len(text)//(seq_len) 

In [22]:
print(f"Total number of sequences within text: {total_num_seq}")

Total number of sequences within text: 45380


### Creating training sequences

We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [23]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

print(type(char_dataset))

<class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>


In [24]:
for item in char_dataset.take(100):
     print(idx_to_char[item.numpy()]) # convert TensorSliceDataset item to NumPy array first 
                                      # in order to print it out



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing.

Here is what ``drop_remainder`` means:

``drop_remainder``: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.

In [25]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"


Sequence Out: "ello my name"

In [26]:
def create_seq_targets(seq):
    input_txt = seq[:-1] # e.g. "Hello my nam"
    target_txt = seq[1:] # e.g. "ello my name"
    return input_txt, target_txt

In [27]:
dataset = sequences.map(create_seq_targets)

In [28]:
type(dataset)

tensorflow.python.data.ops.map_op._MapDataset

In [29]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(idx_to_char[input_txt.numpy()]))
    print()
    print(target_txt.numpy())
    print("".join(idx_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [30]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [31]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

## Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

In [33]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [34]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [35]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [36]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    
    model = Sequential()
    
    model.add(
        Embedding(
            vocab_size, 
            embed_dim, 
            batch_input_shape=[batch_size, None]
        )
    )
    
    model.add(
        GRU(
            rnn_neurons, 
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'
        )
    )
    
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [37]:
model = create_model(
                    vocab_size = vocab_size,
                    embed_dim=embed_dim,
                    rnn_neurons=rnn_neurons,
                    batch_size=batch_size
        )

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.

In [39]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [40]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[-1.38281577e-03, -1.20214291e-03, -2.84469174e-03, ...,
         -3.28585948e-03, -5.33046201e-03,  6.46069937e-04],
        [ 1.99546595e-03,  3.52930021e-03, -1.24996249e-03, ...,
         -6.13524998e-03, -1.26291648e-03, -1.83680654e-03],
        [-4.49904427e-03,  1.65677851e-03, -6.90698065e-03, ...,
         -3.06714233e-03,  4.22158366e-04,  1.02609338e-03],
        ...,
        [-6.71130093e-03, -4.07097815e-03, -8.85701645e-03, ...,
          1.50741939e-03,  5.60282683e-03,  1.74349837e-03],
        [-7.27473246e-03, -4.71742172e-03, -9.23874695e-03, ...,
          2.08955375e-03,  6.85420819e-03,  1.50956318e-03],
        [ 3.66707100e-03, -8.60795099e-03, -7.81746395e-03, ...,
          8.36499501e-03,  9.60237440e-03,  6.31402992e-03]],

       [[-5.13854017e-03, -1.01270818e-03, -6.35854062e-03, ...,
          1.10566740e-04,  1.36299594e-03,  1.38577016e-03],
        [-3.88838304e-03,  2.16523977e-03,  1.8

In [41]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [42]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[17],
       [65],
       [54],
       [ 0],
       [70],
       [25],
       [43],
       [74],
       [67],
       [65],
       [74],
       [ 1],
       [51],
       [42],
       [81],
       [35],
       [81],
       [30],
       [61],
       [43],
       [66],
       [46],
       [78],
       [23],
       [ 5],
       [ 5],
       [11],
       [67],
       [60],
       [11],
       [25],
       [45],
       [37],
       [81],
       [56],
       [ 3],
       [46],
       [24],
       [66],
       [70],
       [16],
       [23],
       [15],
       [51],
       [82],
       [11],
       [52],
       [80],
       [29],
       [38],
       [21],
       [79],
       [28],
       [48],
       [15],
       [49],
       [57],
       [10],
       [83],
       [21],
       [32],
       [19],
       [50],
       [43],
       [54],
       [59],
       [81],
       [10],
       [62],
       [19],
       [59],
       [69],
       [38],
   

In [43]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [44]:
sampled_indices

array([17, 65, 54,  0, 70, 25, 43, 74, 67, 65, 74,  1, 51, 42, 81, 35, 81,
       30, 61, 43, 66, 46, 78, 23,  5,  5, 11, 67, 60, 11, 25, 45, 37, 81,
       56,  3, 46, 24, 66, 70, 16, 23, 15, 51, 82, 11, 52, 80, 29, 38, 21,
       79, 28, 48, 15, 49, 57, 10, 83, 21, 32, 19, 50, 43, 54, 59, 81, 10,
       62, 19, 59, 69, 38, 73, 60, 61, 36, 49, 78, 63, 16, 47, 16, 66,  4,
       55, 46, 12, 10, 36, 16, 47, 25, 82, 80, 60, 74, 49, 50, 29,  3, 51,
       50, 83, 46, 77, 11, 50, 15, 41, 68, 56,  7, 20, 18, 31, 53,  2, 68,
       22])

In [45]:
print("Given the input seq: \n")
print("".join(idx_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(idx_to_char[sampled_indices ]))

Given the input seq: 

if he does think
    He had not my virginity.
  KING. What say'st thou to her?
  BERTRAM. She's impudent, my lord,
    A


Next Char Predictions: 

6j_
o?Rsljs ZQzJzEfRkUw<''0le0?TLza"U>ko5<4Z|0[yDM:xCW4Xb.}:G8YR_dz.g8dnMrefKXwh5V5k&`U1.K5V?|yesXYD"ZY}Uv0Y4Pma)97F]!m;


After confirming the dimensions are working, let's train our network!

In [46]:
epochs=30

In [47]:
# model.fit(dataset,epochs=epochs)
# model.save('shakespeare_gen.h5') 

## Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a ``batch_size=1``. We can create a new model with this batch size, then load our saved models weights. Then call ``.build()`` on the model:

In [48]:
from tensorflow.keras.models import load_model

In [49]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''

    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_idx[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected

    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):

        # Generate Predictions
        predictions = model(input_eval)

        # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Transform back to character letter
        text_generated.append(idx_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [52]:
print(generate_text(model,"flower",gen_size=1000))

flower of these largess,
    And in these many yielding ransom by th' idleness
    Perforce my wife's in such discomfion;
    No marvel hous'd, and therefore away,
    So soon as I am not your false-taken,
    An if thou drew her kinsman's train, that duty seens,
    And care noto for the rich part of man
    And gibestime had so much content with a most
    end on his wish she been bleeding in his tider. Dies, the mass, four
    brother, he hath not Am'd upon a pagienking that?
  DIOMEDES. No, I will rather kill your brother cry,
    No, not that I have been so faith and pretty!
    O, let him come under your lord?
    Where's Run away, sir? This is our own.
  LAUNCE. Who would excuse yourself;
    To bring a ravenous answer in the waters,
    Because what beastly pines drinks matter in these woe?
    Am freely spoke of, neither know thou call'd?
    The Circh, five flouring of me. What your pleasure.
    I am as long as to this none of yours?
  Jul. She is not meet, and one forfeit t